In [1]:
import papermill as pm
import scrapbook as sb
import pandas as pd
from datetime import datetime, timedelta
import os

C:\Users\Tonijua\anaconda3\envs\mfe-preprogram\lib\site-packages\papermill\iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


In [2]:
# Connect to database to find last 2 complete weeks
connection_string = 'sqlite:///D://UCB//Pre-MFE 2022//Python//UCB-MFE-python-preprogram//Lectures//Lecture 4//data//data.db'
prices = pd.read_sql(f'''
SELECT 
    * 
FROM ohlc 
''', connection_string)

prices['ts'] = pd.to_datetime(prices['ts'])
prices.set_index('ts', inplace=True)
prices.index

DatetimeIndex(['2021-11-01 00:00:00', '2021-11-01 01:00:00',
               '2021-11-01 02:00:00', '2021-11-01 03:00:00',
               '2021-11-01 04:00:00', '2021-11-01 05:00:00',
               '2021-11-01 06:00:00', '2021-11-01 07:00:00',
               '2021-11-01 08:00:00', '2021-11-01 09:00:00',
               ...
               '2021-12-14 15:00:00', '2021-12-14 16:00:00',
               '2021-12-14 17:00:00', '2021-12-14 18:00:00',
               '2021-12-14 19:00:00', '2021-12-14 20:00:00',
               '2021-12-14 21:00:00', '2021-12-14 22:00:00',
               '2021-12-14 23:00:00', '2021-12-15 00:00:00'],
              dtype='datetime64[ns]', name='ts', length=11627, freq=None)

We see the last date is 2021-12-14, so we run our template for the last 2 complete weeks previous to that point

In [3]:
def last_dow(d: datetime, dow: int) -> datetime:
    '''
    Gets the date of the last day of week, using datetime convention of Monday is 0 and Sunday is 6
    '''
    dow_d = d.weekday()
    days_delta = (7 - dow) + dow_d if dow_d < dow else dow_d - dow
    return d - timedelta(days=days_delta)

In [5]:
begin_date = (last_dow(datetime(2021, 12, 14), 4) - timedelta(days=11))
end_date = last_dow(datetime(2021, 12, 14), 4) + timedelta(days=1)
print(begin_date)
print(end_date)

2021-11-29 00:00:00
2021-12-11 00:00:00


In [6]:
# We get all the tokens in database
tokens = prices.token.unique()
tokens

array(['BTC', 'ETH', 'USDT', 'SOL', 'ADA', 'DOT', 'AVAX', 'ATOM', 'CRV',
       'AAVE', 'COMP'], dtype=object)

In [7]:
# check the last date is the same for all tokens
for token in tokens:
    print(token, prices[prices.token == token].index[-1])

BTC 2021-12-15 00:00:00
ETH 2021-12-15 00:00:00
USDT 2021-12-15 00:00:00
SOL 2021-12-15 00:00:00
ADA 2021-12-15 00:00:00
DOT 2021-12-15 00:00:00
AVAX 2021-12-15 00:00:00
ATOM 2021-12-15 00:00:00
CRV 2021-12-15 00:00:00
AAVE 2021-12-15 00:00:00
COMP 2021-12-15 00:00:00


In [8]:
# create output folder
base_output_folder = os.path.expanduser('outputs')

if not os.path.exists(base_output_folder):
        os.makedirs(base_output_folder)

In [9]:
# run the template notebook across all tokens in the database and the last 2 complete weeks in the database.
for token in tokens:
    print(f'running for {token} for date {begin_date} to {end_date}', end='\r')
    res = pm.execute_notebook(
        'homework4_template.ipynb',
        f'{base_output_folder}/homework4_{token}_{begin_date.strftime("%Y-%m-%d-%H-%M-%S")}.ipynb',
        parameters = {
            'from_date': f'{begin_date}',
            'to_date': f'{end_date}',
            'token': token,
            'connection_string': connection_string,
        }
    )

Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

...

Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

Summarize the following in a single DataFrame in the runner notebook
- avg hourly return by run
- avg volatility by run
- maximum drawdown over the entire period of each run
- range of last close price - first close price

In [10]:
nbs = sb.read_notebooks('outputs')

In [13]:
pd.DataFrame([
    pd.Series({
        'token': scrap['token'].data,
        'from_date': scrap['from_date'].data,
        'to_date': scrap['to_date'].data,
        'avg hourly return (%)': scrap['prices'].data.hourly_return.mean()*100,
        'avg volatility': scrap['prices'].data.volatility.mean(),
        'max drawdown (%)': scrap['prices'].data.max_drawdown.max()*100,
        'range of last close price - first close price': scrap['prices'].data.close[-1] - scrap['prices'].data.close[0],
    }) 
    for book, scrap in nbs.notebook_scraps.items()
])

,token,from_date,to_date,avg hourly return (%),avg volatility,max drawdown (%),range of last close price - first close price
0,AAVE,2021-11-29 00:00:00,2021-12-11 00:00:00,-0.108586,0.222245,-2.458423,-67.8850
1,ADA,2021-11-29 00:00:00,2021-12-11 00:00:00,-0.080699,0.221098,-3.048095,-0.3555
2,ATOM,2021-11-29 00:00:00,2021-12-11 00:00:00,-0.044326,0.311034,-1.478084,-4.2860
3,AVAX,2021-11-29 00:00:00,2021-12-11 00:00:00,-0.081673,0.302500,-3.480506,-26.8300
4,BTC,2021-11-29 00:00:00,2021-12-11 00:00:00,-0.061693,0.161354,-2.460411,-9778.9100
5,COMP,2021-11-29 00:00:00,2021-12-11 00:00:00,-0.139563,0.233596,-1.920344,-99.6500
6,CRV,2021-11-29 00:00:00,2021-12-11 00:00:00,-0.105843,0.344594,-3.384402,-1.4662
7,DOT,2021-11-29 00:00:00,2021-12-11 00:00:00,-0.089501,0.253899,-2.939946,-8.9050
8,ETH,2021-11-29 00:00:00,2021-12-11 00:00:00,-0.024270,0.179052,-1.822069,-337.0600
9,SOL,2021-11-29 00:00:00,2021-12-11 00:00:00,-0.046930,0.254311,-3.003465,-29.7070
